<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Downloading model fields using netCDF Subset Service (NCSS)</h1>
<h3>Unidata Python Workshop</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

<div style="float:right; width:250 px"><img src="https://unidata.github.io/siphon/latest/_images/tds-logo.png" alt="TDS" style="height: 200px;"></div>

## Overview:

* **Teaching:** 15 minutes
* **Exercises:** 15 minutes

### Questions
1. What is the netCDF Subset Service (NCSS)?
1. How can I use Siphon to make an NCSS request?
1. How do I plot gridded fields using CartoPy?

### Objectives
1. <a href="#ncss">Use siphon to make a request using NCSS</a>
1. <a href="#projection">Making sense of netCDF</a>
1. <a href="#plotting">Plot on a map</a>
1. <a href="#pointrequest">Requesting for a single point</a>

<a name="ncss"></a>
## 1. What is NCSS?

In [ ]:
# Resolve the latest HRRR dataset
from siphon.catalog import TDSCatalog

# Set up access via NCSS
gfs_catalog = ('http://thredds-test.unidata.ucar.edu/thredds/catalog/casestudies/'
               'irma/model/gfs/catalog.xml?dataset=casestudies/irma/model/gfs/Best')
cat = TDSCatalog(gfs_catalog)
ncss = cat.datasets[0].subset()

We can see what variables are available from ncss as well:

In [ ]:
ncss.variables

From here, we can build a query to ask for the data we want from the server.

In [ ]:
from datetime import datetime

# Create a new NCSS query
query = ncss.query()

# Request data in netCDF4 format
query.accept('netcdf4')

# Ask for our variable
query.variables('Temperature_isobaric')

# Ask for the 500 hPa surface
query.vertical_level(50000)

# Set the time range of data we want
query.time_range(datetime(2017, 9, 5), datetime(2017, 9, 6))

# Set the spatial limits
query.lonlat_box(west=-110, east=-45, north=50, south=10)

# get the data!
data = ncss.get_data(query)

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="projection"></a>
## 2. Making sense of netCDF

In [ ]:
data

In [ ]:
ncvar = data.variables['Temperature_isobaric']
ncvar

We need to find the right variable for time (the GRIB collections can have multiple). To do so generally, we need to look at the `coordinates` attribute to see what the correct name of the time variable is.

In [ ]:
# Find the correct time dimension name
for coord in ncvar.coordinates.split():
    if 'time' in coord:
        timevar = data.variables[coord]
        break
timevar

In [ ]:
timevar[:]

We need to convert the time variable to `datetime`s. We can use `num2date` to handle this for us:

In [ ]:
from netCDF4 import num2date

time = num2date(timevar[:], timevar.units)
time[6]

Also need to pull out lon/lat:

In [ ]:
longitude = data.variables['longitude'][:]
latitude = data.variables['latitude'][:]

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="plotting"></a>
## Visualize the grid

In [ ]:
# Get our state borders
import cartopy.feature as cfeat

states_provinces = cfeat.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lakes',
        scale='50m',
        facecolor='none')

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

# GFS uses lon/lat grid
data_projection = ccrs.PlateCarree()

# Make it easy to change what time step we look at
t_step = 8

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.LambertConformal())
mesh = ax.pcolormesh(longitude, latitude, ncvar[t_step].squeeze(),
                     transform=data_projection, zorder=0)

# add some common geographic features
ax.coastlines(resolution='10m', color='black', zorder=1)
ax.add_feature(states_provinces, edgecolor='black', zorder=1)
ax.add_feature(cfeat.BORDERS)

# add some lat/lon gridlines
ax.gridlines()

# add a colorbar
cax = fig.colorbar(mesh)
cax.set_label(ncvar.units)

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
        <li>Extend the plot above by plotting contours of 500 hPa geopotential heights</li>
        <li>Add a title to the plot with the correct time</li>
    </ul>
</div>

In [ ]:
# Set up an NCSS query from thredds using siphon
query = ncss.query()

#
# SET UP QUERY
#

# Download data using NCSS
#ncss.get_data(query)

data_projection = ccrs.PlateCarree()

# Plot using CartoPy and Matplotlib
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.LambertConformal())

#
# YOUR PLOT HERE
#

# add some common geographic features
ax.coastlines(resolution='10m', color='black', zorder=1)
ax.add_feature(states_provinces, edgecolor='black', zorder=1)
ax.add_feature(cfeat.BORDERS)

# add some lat/lon gridlines
ax.gridlines()

In [ ]:
# %load solutions/model_plot.py


<a href="#top">Top</a>
<hr style="height:2px;">

<a name="pointrequest"></a>
## 4. NCSS Point Request
We can also request data for a specfic lon/lat point, across vertical coordinates or times.

In [ ]:
cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/'
                 'Global_0p5deg/catalog.xml?dataset=grib/NCEP/GFS/Global_0p5deg/Best')
ncss = cat.datasets[0].subset()

point_query = ncss.query()
point_query.time(datetime.utcnow())
point_query.accept('netcdf4')
point_query.variables('Temperature_isobaric', 'Relative_humidity_isobaric')
point_query.variables('u-component_of_wind_isobaric', 'v-component_of_wind_isobaric')
point_query.lonlat_point(-101.877, 33.583)

# get the data!
point_data = ncss.get_data(point_query)

Skew-T diagrams typical use specific units. First, let's assign units to the variables we requested:

In [ ]:
from metpy.units import units

# get netCDF variables
pressure = point_data.variables["isobaric"]
temp = point_data.variables["Temperature_isobaric"]
u_cmp = point_data.variables["u-component_of_wind_isobaric"]
v_cmp = point_data.variables["v-component_of_wind_isobaric"]
relh = point_data.variables["Relative_humidity_isobaric"]

# download data and assign the units based on the variables metadata
p = pressure[:].squeeze() * units(pressure.units)
T = temp[:].squeeze() * units(temp.units)
u = u_cmp[:].squeeze() * units(u_cmp.units)
v = v_cmp[:].squeeze() * units(v_cmp.units)
relh = relh[:].squeeze() * units('percent')

We also need to calculate dewpoint from our relative humidity data:

In [ ]:
import metpy.calc as mpcalc

Td = mpcalc.dewpoint_rh(T, relh)

Now, let's change those units to what we typically see used in Skew-T diagrams

In [ ]:
p = p.to(units.millibar)
T = T.to(units.degC)
Td = Td.to(units.degC)
u = u.to(units.knot)
v = v.to(units.knot)

In [ ]:
from metpy.calc import lcl, dry_lapse, parcel_profile
from metpy.plots import SkewT
from metpy.units import concatenate

# Create a new figure. The dimensions here give a good aspect ratio
fig = plt.figure(figsize=(9, 9))
skew = SkewT(fig, rotation=45)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(p, T, 'tab:red')
skew.plot(p, Td, 'tab:green')
skew.plot_barbs(p, u, v)
skew.ax.set_ylim(1000, 100)
skew.ax.set_xlim(-40, 60)

# Add the relevant special lines
skew.plot_dry_adiabats()
skew.plot_moist_adiabats()
skew.plot_mixing_lines()